# 图片分成六等分

In [ ]:
from PIL import Image
import os
import random
from tqdm import tqdm

# 根目录
root_dir = "/data_lg/wentai_share/mvhuman/train/"
# 指定结果输出文件夹
output_dir = "/data_lg/keru/DP_gpt_250502/mask"

# 存储所有图片路径的列表
image_paths = []

# 遍历根目录下的所有文件夹
for subject_id in os.listdir(root_dir):
    subject_dir = os.path.join(root_dir, subject_id)
    if os.path.isdir(subject_dir):
        id_maps_dir = os.path.join(subject_dir, "id_maps")
        if os.path.isdir(id_maps_dir):
            cloth_map_path = os.path.join(id_maps_dir, "cloth_mask.png")
            if os.path.isfile(cloth_map_path):
                image_paths.append((cloth_map_path, subject_id))

# 使用 tqdm 显示进度条
for path, subject_id in tqdm(image_paths, desc="处理图片", unit="张"):
    img = Image.open(path)
    width, height = img.size

    # 为每个 subject id 创建子文件夹
    subject_output_dir = os.path.join(output_dir, subject_id)
    os.makedirs(subject_output_dir, exist_ok=True)

    # 计算每部分的宽度
    part_width = width // 6

    # 进行六等分并保存
    for i in range(6):
        left = i * part_width
        upper = 0
        right = left + part_width
        lower = height

        part = img.crop((left, upper, right, lower))
        part_filename = f"part_{i}.png"
        part_path = os.path.join(subject_output_dir, part_filename)
        part.save(part_path)
    

# 根据mask 给原图换底色换成白色

In [ ]:
from PIL import Image
import os
from tqdm import tqdm

# result 文件夹路径
result_dir = "/data_lg/keru/DP_gpt_250502/result"
# mask 文件夹路径
mask_dir = "/data_lg/keru/DP_gpt_250502/mask"

# 遍历 result 文件夹中的每个 subject id 文件夹
subject_dirs = [d for d in os.listdir(result_dir) if os.path.isdir(os.path.join(result_dir, d))]
for subject_dir in tqdm(subject_dirs, desc="处理文件夹", unit="个"):
    subject_result_path = os.path.join(result_dir, subject_dir)
    subject_mask_path = os.path.join(mask_dir, subject_dir)

    # 检查 subject id 在 mask 文件夹中是否存在
    if not os.path.exists(subject_mask_path):
        print(f"警告: {subject_mask_path} 不存在，跳过该文件夹。")
        continue

    # 遍历 subject id 文件夹中的每个图片
    image_files = [f for f in os.listdir(subject_result_path) if f.endswith('.png')]
    for filename in tqdm(image_files, desc=f"处理 {subject_dir} 文件夹内图片", unit="张"):
        result_img_path = os.path.join(subject_result_path, filename)
        mask_img_path = os.path.join(subject_mask_path, filename)

        if not os.path.exists(mask_img_path):
            print(f"警告: {mask_img_path} 不存在，跳过此图片。")
            continue

        try:
            # 打开 result 图片和 mask 图片
            result_img = Image.open(result_img_path).convert('RGBA')
            mask_img = Image.open(mask_img_path).convert('L')

            width, height = result_img.size
            for x in range(width):
                for y in range(height):
                    # 获取 mask 图对应位置的像素值
                    mask_pixel = mask_img.getpixel((x, y))
                    # 如果 mask 图该位置像素值为黑色（通常为 0）
                    if mask_pixel == 0:
                        # 将 result 图对应位置像素替换为白色
                        result_img.putpixel((x, y), (255, 255, 255, 255))

            # 保存处理后的图片，替换原文件
            result_img.save(result_img_path)
        except Exception as e:
            print(f"处理 {result_img_path} 时出错: {e}")
    

# 挑选valid数据

In [ ]:
import os
import shutil

def copy_valid_folders(valid_ids_file, source_dir, target_dir):
    # 读取valid_ids.txt中的ID列表
    with open(valid_ids_file, 'r') as f:
        valid_ids = [line.strip() for line in f.readlines()]

    # 确保目标文件夹存在
    os.makedirs(target_dir, exist_ok=True)

    # 遍历每个valid_id
    for valid_id in valid_ids:
        source_folder = os.path.join(source_dir, valid_id)
        
        # 确保源文件夹存在
        if os.path.isdir(source_folder):
            target_folder = os.path.join(target_dir, valid_id)
            os.makedirs(target_folder, exist_ok=True)

            # 复制文件夹及其中的所有文件
            try:
                for item in os.listdir(source_folder):
                    s = os.path.join(source_folder, item)
                    t = os.path.join(target_folder, item)
                    if os.path.isdir(s):
                        shutil.copytree(s, t)
                    else:
                        shutil.copy2(s, t)
                print(f"Copied {valid_id} to {target_folder}")
            except Exception as e:
                print(f"Error copying {valid_id}: {e}")
        else:
            print(f"Source folder for {valid_id} does not exist: {source_folder}")

# Example Usage
valid_ids_file = "/data_lg/keru/DP_gpt_250502/valid_ids.txt"  # valid_ids.txt 文件路径
source_dir = "/data_lg/keru/DP_gpt_250502/raw_result"  # 源文件夹路径
target_dir = "/data_lg/keru/DP_gpt_250502/valid_result"  # 目标文件夹路径

# 调用函数
copy_valid_folders(valid_ids_file, source_dir, target_dir)


In [10]:
# 统计 /data_lg/keru/DP_gpt_250502/valid_result 文件夹中的所有文件数量
# 包括文件和子文件夹内的文件
import os
target_dir = "/data_lg/keru/DP_gpt_250502/valid_result"

# 确保文件夹存在
if os.path.exists(target_dir):
    all_files = []
    for root, dirs, files in os.walk(target_dir):
        all_files.extend(files)  # 统计所有文件
    # 输出文件数量
    print(f"Total number of files: {len(all_files)}")  # 使用 print 输出结果
else:
    print(f"The folder {target_dir} does not exist.")



Total number of files: 20638
